In [15]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
from pathlib import Path
import os

In [16]:
# Setup paths
import sys
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version}")

# List files from Path API
all_dirs = list(Path('.').glob('**/*BraTS-GLI-00006*'))
print(f"Found BraTS dirs via glob: {all_dirs[:3]}")

# Direct file check
test_path = Path(r'D:\study\licenta\creier\dataset\BRATS\BraTS2024-BraTS-GLI-TrainingData\training_data1_v2\BraTS-GLI-00006-100\BraTS-GLI-00006-100-t1c.nii.gz')
print(f"\nDirect check: {test_path}")
print(f"File exists: {test_path.exists()}")

# Use specific patient
patient_ids = ['BraTS-GLI-00006-100', 'BraTS-GLI-00006-101']

print(f"\n📌 Selected patient: BraTS-GLI-00006")
print(f"   Scans: {patient_ids}")

Python executable: /usr/bin/python3
Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Found BraTS dirs via glob: []

Direct check: D:\study\licenta\creier\dataset\BRATS\BraTS2024-BraTS-GLI-TrainingData\training_data1_v2\BraTS-GLI-00006-100\BraTS-GLI-00006-100-t1c.nii.gz
File exists: False

📌 Selected patient: BraTS-GLI-00006
   Scans: ['BraTS-GLI-00006-100', 'BraTS-GLI-00006-101']


In [9]:
# Load all 4 modalities for both scans
def load_brats_patient(patient_id, brats_root):
    """Load all 4 MRI sequences and segmentation for a BraTS patient"""
    patient_dir = brats_root / patient_id
    
    print(f"Loading from: {patient_dir}")
    print(f"Directory exists: {patient_dir.exists()}")
    
    modalities = {
        't1n': f'{patient_id}-t1n.nii.gz',  # T1 native
        't1c': f'{patient_id}-t1c.nii.gz',  # T1 with contrast
        't2w': f'{patient_id}-t2w.nii.gz',  # T2 weighted
        't2f': f'{patient_id}-t2f.nii.gz',  # T2-FLAIR
        'seg': f'{patient_id}-seg.nii.gz'   # Segmentation
    }
    
    data = {}
    for mod_name, filename in modalities.items():
        filepath = patient_dir / filename
        if filepath.exists():
            img = nib.load(str(filepath))
            data[mod_name] = img.get_fdata()
            print(f"  ✅ {filename}")
        else:
            print(f"  ❌ {filename} not found at {filepath}")
    
    return data

# Load data for both scans
all_scans_data = {}
for patient_id in patient_ids:
    all_scans_data[patient_id] = load_brats_patient(patient_id, brats_root)

if all_scans_data[patient_ids[0]]:
    print(f"\n📏 Shapes: {all_scans_data[patient_ids[0]]['t1c'].shape}")

Loading from: D:\study\licenta\creier\dataset\BRATS\BraTS2024-BraTS-GLI-TrainingData\training_data1_v2/BraTS-GLI-00006-100
Directory exists: False
  ❌ BraTS-GLI-00006-100-t1n.nii.gz not found at D:\study\licenta\creier\dataset\BRATS\BraTS2024-BraTS-GLI-TrainingData\training_data1_v2/BraTS-GLI-00006-100/BraTS-GLI-00006-100-t1n.nii.gz
  ❌ BraTS-GLI-00006-100-t1c.nii.gz not found at D:\study\licenta\creier\dataset\BRATS\BraTS2024-BraTS-GLI-TrainingData\training_data1_v2/BraTS-GLI-00006-100/BraTS-GLI-00006-100-t1c.nii.gz
  ❌ BraTS-GLI-00006-100-t2w.nii.gz not found at D:\study\licenta\creier\dataset\BRATS\BraTS2024-BraTS-GLI-TrainingData\training_data1_v2/BraTS-GLI-00006-100/BraTS-GLI-00006-100-t2w.nii.gz
  ❌ BraTS-GLI-00006-100-t2f.nii.gz not found at D:\study\licenta\creier\dataset\BRATS\BraTS2024-BraTS-GLI-TrainingData\training_data1_v2/BraTS-GLI-00006-100/BraTS-GLI-00006-100-t2f.nii.gz
  ❌ BraTS-GLI-00006-100-seg.nii.gz not found at D:\study\licenta\creier\dataset\BRATS\BraTS2024-BraTS

In [8]:
def get_middle_slice(data_shape):
    """Get the middle slice in Z axis"""
    return data_shape[2] // 2

def normalize_slice(slice_2d):
    """Normalize a 2D slice for display"""
    p2, p98 = np.percentile(slice_2d[slice_2d > 0], [2, 98]) if np.any(slice_2d > 0) else (0, 1)
    normalized = (slice_2d - p2) / (p98 - p2 + 1e-6)
    return np.clip(normalized, 0, 1)

# Get middle slices for all data
middle_slices = {}
for patient_id, data in all_scans_data.items():
    middle_idx = get_middle_slice(data['t1c'].shape)
    middle_slices[patient_id] = middle_idx
    print(f"{patient_id}: middle slice = {middle_idx}")

KeyError: 't1c'

In [ ]:
# Create comprehensive visualization for both scans
fig = plt.figure(figsize=(18, 12))
gs = fig.add_gridspec(3, 4, hspace=0.35, wspace=0.3)
fig.suptitle('BraTS-GLI-00006: Complete MRI & Segmentation Analysis', fontsize=18, fontweight='bold', y=0.995)

for scan_idx, patient_id in enumerate(patient_ids):
    data = all_scans_data[patient_id]
    middle_idx = middle_slices[patient_id]
    
    scan_label = patient_id.split('-')[-1]  # Get "100" or "101"
    
    # ===== ROW 1: All 4 MRI Modalities =====
    modalities_info = [
        ('t1n', 'T1 Native'),
        ('t1c', 'T1 + Contrast'),
        ('t2w', 'T2 Weighted'),
        ('t2f', 'T2-FLAIR')
    ]
    
    for mod_idx, (mod_key, mod_label) in enumerate(modalities_info):
        ax = fig.add_subplot(gs[0, scan_idx * 2 + (mod_idx // 2)])
        
        if mod_key in data:
            img_slice = data[mod_key][:, :, middle_idx]
            img_norm = normalize_slice(img_slice)
            
            im = ax.imshow(img_slice, cmap='gray')
            ax.set_title(f'Scan {scan_label} - {mod_label}', fontsize=11, fontweight='bold')
        else:
            ax.text(0.5, 0.5, f'{mod_key} not found', ha='center', va='center', transform=ax.transAxes)
            ax.set_title(f'Scan {scan_label} - {mod_label}', fontsize=11, fontweight='bold')
        
        ax.axis('off')
    
    # ===== ROW 2: Segmentation Mask =====
    ax2 = fig.add_subplot(gs[1, scan_idx * 2])
    if 'seg' in data:
        seg_slice = data['seg'][:, :, middle_idx]
        
        # Create colored segmentation visualization
        seg_colored = np.zeros((*seg_slice.shape, 3))
        
        # Necrotic core (label 1) - RED
        seg_colored[seg_slice == 1] = [1, 0, 0]
        # Edema (label 2) - BLUE
        seg_colored[seg_slice == 2] = [0, 0, 1]
        # Enhancing (label 4) - GREEN
        seg_colored[seg_slice == 4] = [0, 1, 0]
        
        ax2.imshow(seg_colored)
        ax2.set_title(f'Scan {scan_label} - Segmentation Mask', fontsize=11, fontweight='bold')
    
    ax2.axis('off')
    
    # ===== ROW 2: Overlay on T1c =====
    ax3 = fig.add_subplot(gs[1, scan_idx * 2 + 1])
    if 'seg' in data and 't1c' in data:
        t1c_slice = data['t1c'][:, :, middle_idx]
        seg_slice = data['seg'][:, :, middle_idx]
        
        # Background: T1c grayscale
        ax3.imshow(t1c_slice, cmap='gray', alpha=0.8)
        
        # Overlay: colored segmentation with transparency
        seg_colored_alpha = np.zeros((*seg_slice.shape, 4))
        seg_colored_alpha[seg_slice == 1] = [1, 0, 0, 0.7]  # Red - Necrotic
        seg_colored_alpha[seg_slice == 2] = [0, 0, 1, 0.6]  # Blue - Edema
        seg_colored_alpha[seg_slice == 4] = [0, 1, 0, 0.7]  # Green - Enhancing
        
        ax3.imshow(seg_colored_alpha)
        ax3.set_title(f'Scan {scan_label} - Overlay on T1c', fontsize=11, fontweight='bold')
    
    ax3.axis('off')
    
    # ===== ROW 3: Statistics & Info =====
    ax4 = fig.add_subplot(gs[2, scan_idx * 2:])
    ax4.axis('off')
    
    if 'seg' in data:
        seg_data = data['seg']
        necrotic = np.sum(seg_data == 1)
        edema = np.sum(seg_data == 2)
        enhancing = np.sum(seg_data == 4)
        total = necrotic + edema + enhancing
        
        stats_text = f"""
        SCAN: {patient_id}
        ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
        
        Segmentation Statistics:
        • Necrotic Core (RED):     {necrotic:,} voxels
        • Edema (BLUE):            {edema:,} voxels  
        • Enhancing Tumor (GREEN): {enhancing:,} voxels
        ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
        • TOTAL TUMOR:             {total:,} voxels
        
        Middle Slice Index: {middle_idx} / {seg_data.shape[2]}
        """
        
        ax4.text(0.05, 0.95, stats_text, transform=ax4.transAxes, 
                fontsize=10, verticalalignment='top', fontfamily='monospace',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))

# Add global legend
legend_elements = [
    mpatches.Patch(facecolor='red', label='Necrotic Core'),
    mpatches.Patch(facecolor='blue', label='Edema'),
    mpatches.Patch(facecolor='green', label='Enhancing Tumor')
]
fig.legend(handles=legend_elements, loc='lower center', ncol=3, fontsize=11, 
          bbox_to_anchor=(0.5, -0.01), frameon=True, fancybox=True)

plt.show()

print("✅ Visualization complete!")